# Classificação de Lesões Cervicais com MobileNet

Este notebook implementa um pipeline completo para classificar imagens de citologia em LSIL (Lesão de Baixo Grau) e HSIL (Lesão de Alto Grau) usando transfer learning com MobileNet.

## 1. Configuração e Imports

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import random

# TensorFlow/Keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import Precision, Recall

# Scikit-learn para métricas
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split

# Configurações
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# Parâmetros do modelo
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS_FASE1 = 10
EPOCHS_FASE2 = 20
INITIAL_LR = 1e-4
FINE_TUNE_LR = 1e-5

print("Configurações e imports concluídos!")
print(f"TensorFlow versão: {tf.__version__}")
print(f"GPU disponível: {tf.config.list_physical_devices('GPU')}")

## 2. Preparação e Reorganização dos Dados

In [ ]:
# Definir caminhos
BASE_DIR = Path("c:/Users/IA/Desktop/Dataset Citologia")
TILE_DIR = BASE_DIR / "Tile"
PROCESSED_DIR = BASE_DIR / "dataset_processado"

# Criar estrutura de diretórios processados
for split in ["train", "validation"]:
    for class_name in ["LSIL", "HSIL"]:
        (PROCESSED_DIR / split / class_name).mkdir(parents=True, exist_ok=True)

print("Estrutura de diretórios criada!")

In [ ]:
# Definir mapeamento de categorias para classes
# Baseado na análise do PDF
LSIL_CATEGORIES = [
    "light_dysplastic",
    "im_Dyskeratotic",
    "im_Koilocytotic",
    "Low squamous intra-epithelial lesion"
]

HSIL_CATEGORIES = [
    "moderate_dysplastic",
    "severe_dysplastic",
    "carcinoma_in_situ",
    "High squamous intra-epithelial lesion",
    "Squamous cell carcinoma"
]

# Categorias a serem ignoradas (Normal/Benigno)
IGNORE_CATEGORIES = [
    "im_Metaplastic",
    "im_Parabasal",
    "im_Superficial-Intermediate",
    "Negative for Intraepithelial malignancy"
]

print("Mapeamento de categorias definido!")

In [ ]:
def collect_images(tile_dir, lsil_categories, hsil_categories, ignore_categories):
    """Coletar todas as imagens e suas respectivas classes"""
    images = []
    
    # Percorrer estrutura de diretórios
    for class_dir in ["HISIL", "LISIL"]:
        class_path = tile_dir / class_dir
        
        if not class_path.exists():
            continue
            
        for dataset_dir in class_path.iterdir():
            if dataset_dir.is_dir():
                for category_dir in dataset_dir.iterdir():
                    if category_dir.is_dir():
                        category_name = category_dir.name
                        
                        # Determinar classe final
                        if category_name in lsil_categories:
                            final_class = "LSIL"
                        elif category_name in hsil_categories:
                            final_class = "HSIL"
                        elif category_name in ignore_categories:
                            print(f"Ignorando categoria: {category_name}")
                            continue
                        else:
                            print(f"Categoria não mapeada: {category_name}")
                            continue
                        
                        # Coletar todas as imagens desta categoria
                        for img_file in category_dir.glob("*"):
                            if img_file.suffix.lower() in ['.jpg', '.jpeg', '.png', '.bmp']:
                                images.append({
                                    'path': str(img_file),
                                    'original_class': class_dir,
                                    'category': category_name,
                                    'final_class': final_class
                                })
    
    return images

# Coletar imagens
all_images = collect_images(TILE_DIR, LSIL_CATEGORIES, HSIL_CATEGORIES, IGNORE_CATEGORIES)
print(f"Total de imagens coletadas: {len(all_images)}")

# Criar DataFrame para análise
df_images = pd.DataFrame(all_images)
print("\nDistribuição por classe final:")
print(df_images['final_class'].value_counts())
print("\nDistribuição por categoria:")
print(df_images['category'].value_counts())

In [ ]:
# Dividir dados em treino e validação
train_df, val_df = train_test_split(
    df_images, 
    test_size=0.2, 
    random_state=RANDOM_SEED,
    stratify=df_images['final_class']
)

print(f"Imagens de treino: {len(train_df)}")
print(f"Imagens de validação: {len(val_df)}")
print(f"\nDistribuição no treino:")
print(train_df['final_class'].value_counts())
print(f"\nDistribuição na validação:")
print(val_df['final_class'].value_counts())

In [ ]:
# Copiar imagens para estrutura processada
def copy_images_to_structure(df, split_name):
    """Copiar imagens para a estrutura de diretórios processada"""
    for _, row in df.iterrows():
        src_path = Path(row['path'])
        dst_dir = PROCESSED_DIR / split_name / row['final_class']
        dst_path = dst_dir / src_path.name
        
        if src_path.exists():
            shutil.copy2(src_path, dst_path)
    
    print(f"Imagens copiadas para {split_name}")

# Copiar imagens
copy_images_to_structure(train_df, "train")
copy_images_to_structure(val_df, "validation")

# Verificar estrutura final
print("\nEstrutura final do dataset processado:")
for split in ["train", "validation"]:
    for class_name in ["LSIL", "HSIL"]:
        path = PROCESSED_DIR / split / class_name
        count = len(list(path.glob("*")))
        print(f"{split}/{class_name}: {count} imagens")

## 3. Visualização das Imagens

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

def show_sample_images(class_name, num_samples=4):
    """Mostrar amostras de imagens de uma classe"""
    train_path = PROCESSED_DIR / "train" / class_name
    image_files = list(train_path.glob("*"))[:num_samples]
    
    fig, axes = plt.subplots(1, num_samples, figsize=(15, 4))
    
    for i, img_path in enumerate(image_files):
        img = Image.open(img_path)
        axes[i].imshow(img)
        axes[i].set_title(f"{class_name} - {img_path.name}")
        axes[i].axis('off')
    
    plt.suptitle(f"Amostras de {class_name}")
    plt.tight_layout()
    plt.show()

# Mostrar amostras
show_sample_images("LSIL", 4)
show_sample_images("HSIL", 4)

## 4. Criação dos Geradores de Dados

In [ ]:
# Criar geradores de dados com aumento de dados para treino
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.2,
    fill_mode='nearest'
)

# Gerador de dados para validação (apenas redimensionamento e normalização)
val_datagen = ImageDataGenerator(rescale=1./255)

# Criar geradores
train_generator = train_datagen.flow_from_directory(
    PROCESSED_DIR / "train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    seed=RANDOM_SEED
)

validation_generator = val_datagen.flow_from_directory(
    PROCESSED_DIR / "validation",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    seed=RANDOM_SEED
)

print("Geradores de dados criados!")
print(f"Classes: {train_generator.class_indices}")
print(f"Passos por época (treino): {len(train_generator)}")
print(f"Passos por época (validação): {len(validation_generator)}")

## 5. Construção do Modelo MobileNet

In [ ]:
def create_mobilenet_model(input_shape=IMG_SIZE + (3,), num_classes=1):
    """Criar modelo MobileNetV2 com transfer learning"""
    
    # Carregar MobileNetV2 pré-treinado (sem o topo)
    base_model = MobileNetV2(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    
    # Congelar as camadas da base
    base_model.trainable = False
    
    # Adicionar camadas customizadas
    model = tf.keras.Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='sigmoid')
    ])
    
    return model, base_model

# Criar modelo
model, base_model = create_mobilenet_model()

# Compilar modelo
model.compile(
    optimizer=Adam(learning_rate=INITIAL_LR),
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(), Recall()]
)

print("Modelo criado e compilado!")
print(f"Número total de camadas: {len(model.layers)}")
print(f"Camadas congeladas: {sum(1 for layer in base_model.layers if not layer.trainable)}")
print(f"Camadas treináveis: {sum(1 for layer in model.layers if layer.trainable)}")

In [ ]:
# Mostrar resumo do modelo
model.summary()

## 6. Fase 1: Treinamento do Topo Customizado

In [ ]:
# Callbacks para o treinamento
checkpoint_path = BASE_DIR / "best_model_fase1.h5"

callbacks_fase1 = [
    ModelCheckpoint(
        str(checkpoint_path),
        monitor='val_loss',
        save_best_only=True,
        mode='min',
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    )
]

print("Iniciando Fase 1: Treinamento do topo customizado...")

history_fase1 = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=EPOCHS_FASE1,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=callbacks_fase1,
    verbose=1
)

print("Fase 1 concluída!")

## 7. Visualização do Progresso da Fase 1

In [ ]:
def plot_training_history(history, title="Histórico de Treinamento"):
    """Plotar histórico de treinamento"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # Plotar perda
    ax1.plot(history.history['loss'], label='Perda de Treino')
    ax1.plot(history.history['val_loss'], label='Perda de Validação')
    ax1.set_title(f'{title} - Perda')
    ax1.set_xlabel('Época')
    ax1.set_ylabel('Perda')
    ax1.legend()
    ax1.grid(True)
    
    # Plotar acurácia
    ax2.plot(history.history['accuracy'], label='Acurácia de Treino')
    ax2.plot(history.history['val_accuracy'], label='Acurácia de Validação')
    ax2.set_title(f'{title} - Acurácia')
    ax2.set_xlabel('Época')
    ax2.set_ylabel('Acurácia')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

plot_training_history(history_fase1, "Fase 1")

## 8. Fase 2: Fine-tuning do Modelo

In [ ]:
# Descongelar as últimas camadas da base para fine-tuning
base_model.trainable = True

# Congelar as primeiras camadas (manter early layers congeladas)
fine_tune_at = 100  # Descongelar a partir da camada 100

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Recompilar com taxa de aprendizado menor
model.compile(
    optimizer=Adam(learning_rate=FINE_TUNE_LR),
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(), Recall()]
)

print(f"Fine-tuning: {len([l for l in base_model.layers if l.trainable])} camadas treináveis")
print(f"Camadas congeladas: {len([l for l in base_model.layers if not l.trainable])}")

In [ ]:
# Callbacks para fine-tuning
checkpoint_path_fase2 = BASE_DIR / "best_model_fase2.h5"

callbacks_fase2 = [
    ModelCheckpoint(
        str(checkpoint_path_fase2),
        monitor='val_loss',
        save_best_only=True,
        mode='min',
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=8,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    )
]

print("Iniciando Fase 2: Fine-tuning...")

history_fase2 = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=EPOCHS_FASE2,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=callbacks_fase2,
    initial_epoch=EPOCHS_FASE1,
    verbose=1
)

print("Fase 2 concluída!")

## 9. Visualização do Progresso Completo

In [ ]:
# Combinar históricos
def combine_histories(history1, history2):
    """Combinar históricos de duas fases de treinamento"""
    combined = {}
    
    for key in history1.history.keys():
        combined[key] = history1.history[key] + history2.history[key]
    
    return combined

combined_history = combine_histories(history_fase1, history_fase2)

# Criar objeto mock para plotagem
class MockHistory:
    def __init__(self, history_dict):
        self.history = history_dict

full_history = MockHistory(combined_history)
plot_training_history(full_history, "Treinamento Completo")

## 10. Avaliação do Modelo

In [ ]:
# Avaliar modelo no conjunto de validação
print("Avaliando modelo no conjunto de validação...")

val_loss, val_accuracy, val_precision, val_recall = model.evaluate(
    validation_generator,
    steps=len(validation_generator),
    verbose=1
)

print(f"\nResultados da Validação:")
print(f"Perda: {val_loss:.4f}")
print(f"Acurácia: {val_accuracy:.4f}")
print(f"Precisão: {val_precision:.4f}")
print(f"Recall: {val_recall:.4f}")

In [ ]:
# Obter predições
print("Obtendo predições...")

y_true = []
y_pred = []
y_pred_proba = []

for i in range(len(validation_generator)):
    batch_X, batch_y = validation_generator[i]
    predictions = model.predict(batch_X, verbose=0)
    
    y_true.extend(batch_y)
    y_pred_proba.extend(predictions.flatten())
    y_pred.extend((predictions > 0.5).astype(int).flatten())

y_true = np.array(y_true)
y_pred = np.array(y_pred)
y_pred_proba = np.array(y_pred_proba)

print(f"Predições obtidas: {len(y_pred)} amostras")

In [ ]:
# Relatório de classificação
print("Relatório de Classificação:")
print(classification_report(y_true, y_pred, target_names=['LSIL', 'HSIL']))

# Matriz de confusão
cm = confusion_matrix(y_true, y_pred)
print(f"\nMatriz de Confusão:")
print(cm)

In [ ]:
# Visualizar matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['LSIL', 'HSIL'], 
            yticklabels=['LSIL', 'HSIL'])
plt.title('Matriz de Confusão')
plt.ylabel('Classe Verdadeira')
plt.xlabel('Classe Predita')
plt.show()

# Calcular métricas adicionais
tn, fp, fn, tp = cm.ravel()
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)  # Same as recall

print(f"\nMétricas Adicionais:")
print(f"Especificidade: {specificity:.4f}")
print(f"Sensibilidade (Recall): {sensitivity:.4f}")

In [ ]:
# Curva ROC
fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)
roc_auc = roc_auc_score(y_true, y_pred_proba)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taxa de Falsos Positivos')
plt.ylabel('Taxa de Verdadeiros Positivos')
plt.title('Curva ROC')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

print(f"Área sob a curva ROC: {roc_auc:.4f}")

## 11. Salvamento do Modelo Final

In [ ]:
# Salvar modelo final
final_model_path = BASE_DIR / "modelo_cervical_cancer_final.h5"
model.save(str(final_model_path))

# Salvar também em formato SavedModel
savedmodel_path = BASE_DIR / "modelo_cervical_cancer_savedmodel"
model.save(str(savedmodel_path))

print(f"Modelo salvo em: {final_model_path}")
print(f"Modelo também salvo em formato SavedModel: {savedmodel_path}")

## 12. Teste de Predição em Nova Imagem

In [ ]:
# Função para predizer uma nova imagem
def predict_image(image_path, model, threshold=0.5):
    """Predizer classe de uma nova imagem"""
    from tensorflow.keras.preprocessing import image
    
    # Carregar e pré-processar imagem
    img = image.load_img(image_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    
    # Fazer predição
    prediction = model.predict(img_array, verbose=0)
    probability = prediction[0][0]
    
    # Determinar classe
    predicted_class = 'HSIL' if probability > threshold else 'LSIL'
    confidence = probability if predicted_class == 'HSIL' else 1 - probability
    
    return predicted_class, confidence, probability

# Testar com uma imagem de exemplo
test_image_path = list((PROCESSED_DIR / "validation" / "HSIL").glob("*"))[0]
predicted_class, confidence, raw_prob = predict_image(test_image_path, model)

print(f"Imagem: {test_image_path.name}")
print(f"Classe predita: {predicted_class}")
print(f"Confiança: {confidence:.3f}")
print(f"Probabilidade bruta (HSIL): {raw_prob:.3f}")

# Mostrar imagem
img = Image.open(test_image_path)
plt.figure(figsize=(6, 6))
plt.imshow(img)
plt.title(f"Predição: {predicted_class} (Confiança: {confidence:.3f})")
plt.axis('off')
plt.show()

## 13. Resumo e Conclusões

In [ ]:
# Criar resumo do treinamento
summary = {
    'Dataset': {
        'Total de imagens': len(df_images),
        'Imagens de treino': len(train_df),
        'Imagens de validação': len(val_df),
        'Classes': list(train_generator.class_indices.keys())
    },
    'Modelo': {
        'Arquitetura': 'MobileNetV2',
        'Transfer Learning': 'Sim (ImageNet)',
        'Fine-tuning': 'Sim'
    },
    'Resultados': {
        'Acurácia de Validação': f"{val_accuracy:.4f}",
        'Precisão': f"{val_precision:.4f}",
        'Recall': f"{val_recall:.4f}",
        'AUC-ROC': f"{roc_auc:.4f}",
        'Especificidade': f"{specificity:.4f}"
    }
}

print("=== RESUMO DO TREINAMENTO ===")
for section, data in summary.items():
    print(f"\n{section}:")
    for key, value in data.items():
        print(f"  {key}: {value}")

print(f"\n=== ARQUIVOS SALVOS ===")
print(f"Modelo final: {final_model_path}")
print(f"Modelo (SavedModel): {savedmodel_path}")
print(f"Melhor modelo fase 1: {checkpoint_path}")
print(f"Melhor modelo fase 2: {checkpoint_path_fase2}")

## Próximos Passos

1. **Teste com dados independentes**: Avaliar o modelo em um conjunto de teste completamente novo
2. **Otimização de hiperparâmetros**: Ajustar learning rate, batch size, arquitetura
3. **Análise de erros**: Examinar casos mal classificados para entender limitações
4. **Validação clínica**: Verificar performance com especialistas médicos
5. **Deploy**: Preparar modelo para uso em produção

**Nota**: Este modelo foi treinado para classificação binária (LSIL vs HSIL). Imagens normais/benignas foram excluídas do treinamento conforme a metodologia estabelecida.